#### Prepare Annotations

```

annotation = {
'filename': img_path,
'width': w,
'height': h,
'ann': {
    'bboxes': bboxes.astype(np.float32),
    'labels': labels.astype(np.int64),
    'bboxes_ignore': bboxes_ignore.astype(np.float32),
    'labels_ignore': labels_ignore.astype(np.int64)
}
}

```

In [29]:
annotations = []

In [30]:
import csv
import numpy as np

line_number = 0
with open('mmdetection_input.csv') as f:
    reader = csv.reader(f, skipinitialspace=True)
    header = next(reader)
    while True:
        try:
            row = next(reader)
            line_number += 1
            row_dict = dict(zip(header, row))
            annotations.append(create_annotation(row_dict))
        except:
            break

In [31]:
class_dict = {'1' : 1, '2' : 2, '3' : 3, '5' : 5, '10' : 10, '36' : 36}

In [32]:
def create_annotation(row_dict):
    annotation = {}
    annotation['filename'] = row_dict['filename']
    annotation['width'] = row_dict['width']
    annotation['height'] = row_dict['height']
    annotation['ann'] = {}
    annotation['ann']['bboxes'] = np.array([
                                             int(row_dict['xmin']),
                                             int(row_dict['ymin']),
                                             int(row_dict['xmax']),
                                             int(row_dict['ymax'])
                                            ])
    try:
        annotation['ann']['labels'] = class_dict[row_dict['num_tokens']]
    except:
        print(row_dict)
    
    return annotation
                                                
        
    

In [33]:
len(annotations)

13570

In [35]:
mmcv.dump(annotations, 'mmdetect_output.pkl')